In [14]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

**`Load Data`**
+ `one-hot` format

In [15]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [16]:
print('X_train.shape :', mnist.train.images.shape)
print('Y_train.shape :', mnist.train.labels.shape)

X_train.shape : (55000, 784)
Y_train.shape : (55000, 10)


**`mini batch gradient descent`**

In [17]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size  # 整除

**`优化程序`**
+ batch_size
+ Neural Network Struct
+ Active Function
+ Cost Function : Cross Entropy
+ 使用其他的优化方式

**`Neural Network Struct`**
+ input  : 784
+ output : 10

In [18]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

**`Dropout Param`**

In [19]:
keep_prob = tf.placeholder(tf.float32)

**`tf.nn.softmax`**()

In [20]:
Weight_L1 = tf.Variable(tf.truncated_normal([784,500], stddev=0.1))  # (input, output)
Biases_L1 = tf.Variable(tf.zeros([500]) + 0.1)
Z_L1 = tf.matmul(x, Weight_L1) + Biases_L1
A_L1 = tf.nn.tanh(Z_L1)

Drop_L1 = tf.nn.dropout(A_L1, keep_prob)

In [21]:
Weight_L2 = tf.Variable(tf.truncated_normal([500,300], stddev=0.1))  # (input, output)
Biases_L2 = tf.Variable(tf.zeros([300]) + 0.1)
Z_L2 = tf.matmul(Drop_L1, Weight_L2) + Biases_L2
A_L2 = tf.nn.tanh(Z_L2)

Drop_L2 = tf.nn.dropout(A_L2, keep_prob)

In [22]:
Weight_L3 = tf.Variable(tf.truncated_normal([300,10], stddev=0.1))  # (input, output)
Biases_L3 = tf.Variable(tf.zeros([10]) + 0.1)
Z_L3 = tf.matmul(Drop_L2, Weight_L3) + Biases_L3
A_L3 = tf.nn.softmax(Z_L3)

Prediction = tf.nn.softmax(Z_L3)

**`tf.nn.softmax_cross_entropy_with_logits`**()

In [23]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=Prediction))

**`Optimizer`**

In [24]:
lr = tf.Variable(0.001) # 1e-3

# optimizer = tf.train.GradientDescentOptimizer(lr)
optimizer = tf.train.AdamOptimizer(lr)

train = optimizer.minimize(loss)

**`accuary`**
+ **tf.argmax**(input, axis=None) : Returns the index with the largest value across axes of a tensor
+ **tf.equal**(x, y, name=None) : Returns the truth value of (x == y) element-wise.
+ **tf.cast**(x, dtype) : Casts a tensor to a new type. / 类型转换
+ **tf.reduce_mean**(input_tensor, axis=None) : Computes the mean of elements across dimensions of a tensor.

In [25]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Prediction, 1))
    # return a bool-list

accuary = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**由于该神经网络过于复杂，容易出现overfitting**
+ 因此使用dropout来避免过拟合
+ 当使用dropout之后
    + test accuracy的上升速度变慢，[ 从 96 -> 97 ]， 收敛速度变慢
    + **train accuracy的准确率没有很高**，**test & train accuracy 相对偏差不是很大**

In [26]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(40):
        sess.run(tf.assign(lr, 0.001 * (0.95 ** epoch)))
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={
                x : batch_xs,
                y : batch_ys,
                keep_prob : 0.83
            })
        #if epoch % 100 == 0:
            # keep_prob
            # 1.0 : 所有神经元都在使用
            
        learning_rate = sess.run(lr)
        
        test_acc = sess.run(accuary, feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels,
            keep_prob : 1.0
        })
        train_acc = sess.run(accuary, feed_dict={
            x:mnist.train.images,
            y:mnist.train.labels,
            keep_prob : 1.0
        })

        print("Iter " + str(epoch) + "  Test Acuuary " + str(test_acc) + "  Train Acuuary " + str(train_acc) + " Learning Rate " + str(learning_rate))

Iter 0  Test Acuuary 0.9432  Train Acuuary 0.9446727 Learning Rate 0.001
Iter 1  Test Acuuary 0.9559  Train Acuuary 0.9608909 Learning Rate 0.00095
Iter 2  Test Acuuary 0.9637  Train Acuuary 0.9693818 Learning Rate 0.0009025
Iter 3  Test Acuuary 0.9664  Train Acuuary 0.9742 Learning Rate 0.000857375
Iter 4  Test Acuuary 0.9697  Train Acuuary 0.9786182 Learning Rate 0.00081450626
Iter 5  Test Acuuary 0.968  Train Acuuary 0.9803636 Learning Rate 0.0007737809
Iter 6  Test Acuuary 0.973  Train Acuuary 0.98325455 Learning Rate 0.0007350919
Iter 7  Test Acuuary 0.9732  Train Acuuary 0.9843091 Learning Rate 0.0006983373
Iter 8  Test Acuuary 0.9733  Train Acuuary 0.9848545 Learning Rate 0.0006634204
Iter 9  Test Acuuary 0.975  Train Acuuary 0.9873091 Learning Rate 0.0006302494
Iter 10  Test Acuuary 0.9756  Train Acuuary 0.9877273 Learning Rate 0.0005987369
Iter 11  Test Acuuary 0.9775  Train Acuuary 0.9894182 Learning Rate 0.0005688001
Iter 12  Test Acuuary 0.9757  Train Acuuary 0.98996365 Lea

KeyboardInterrupt: 